# 論文 16：用於關係推理的簡單神經網路模組
## Adam Santoro, David Raposo, David G.T. Barrett 等人，DeepMind (2017)

### 關係網路（Relation Networks, RN）

即插即用的模組，用於推理物件之間的關係。關鍵洞察：明確計算成對關係！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

np.random.seed(42)

## 關係網路架構

核心思想：
```
RN(O) = f_φ( Σ_{i,j} g_θ(o_i, o_j, q) )
```

- **g_θ**：關係函數（處理成對）
- **f_φ**：聚合函數（處理關係）
- **O**：物件集合
- **q**：查詢/上下文

In [ ]:
def relu(x):
    return np.maximum(0, x)

class MLP:
    """簡單的多層感知器"""
    def __init__(self, input_dim, hidden_dims, output_dim):
        self.layers = []
        
        # 建立層
        dims = [input_dim] + hidden_dims + [output_dim]
        for i in range(len(dims) - 1):
            W = np.random.randn(dims[i+1], dims[i]) * 0.01
            b = np.zeros((dims[i+1], 1))
            self.layers.append((W, b))
    
    def forward(self, x):
        """MLP 的前向傳遞"""
        if len(x.shape) == 1:
            x = x.reshape(-1, 1)
        
        for i, (W, b) in enumerate(self.layers):
            x = np.dot(W, x) + b
            # 除最後一層外都用 ReLU
            if i < len(self.layers) - 1:
                x = relu(x)
        
        return x.flatten()

# 測試 MLP
mlp = MLP(input_dim=10, hidden_dims=[20, 20], output_dim=5)
test_input = np.random.randn(10)
output = mlp.forward(test_input)
print(f"MLP 輸出形狀：{output.shape}")

## 關係網路模組

In [ ]:
class RelationNetwork:
    """
    用於推理物件關係的關係網路
    
    RN(O) = f_φ( Σ_{i,j} g_θ(o_i, o_j, q) )
    """
    def __init__(self, object_dim, query_dim, g_hidden_dims, f_hidden_dims, output_dim):
        """
        object_dim：每個物件表示的維度
        query_dim：查詢/問題的維度
        g_hidden_dims：g_θ（關係函數）的隱藏維度
        f_hidden_dims：f_φ（聚合函數）的隱藏維度
        output_dim：最終輸出維度
        """
        # g_θ：處理物件對 + 查詢
        g_input_dim = object_dim * 2 + query_dim
        g_output_dim = g_hidden_dims[-1] if g_hidden_dims else 256
        self.g_theta = MLP(g_input_dim, g_hidden_dims[:-1], g_output_dim)
        
        # f_φ：處理聚合的關係
        f_input_dim = g_output_dim
        self.f_phi = MLP(f_input_dim, f_hidden_dims, output_dim)
    
    def forward(self, objects, query):
        """
        objects：物件表示的列表（每個是一個向量）
        query：查詢/上下文向量
        
        返回：輸出向量
        """
        n_objects = len(objects)
        
        # 計算所有成對的關係
        relations = []
        
        for i in range(n_objects):
            for j in range(n_objects):
                # 串接物件對 + 查詢
                pair_input = np.concatenate([objects[i], objects[j], query])
                
                # 應用 g_θ 計算關係
                relation = self.g_theta.forward(pair_input)
                relations.append(relation)
        
        # 聚合關係（求和）
        aggregated = np.sum(relations, axis=0)
        
        # 應用 f_φ 得到最終輸出
        output = self.f_phi.forward(aggregated)
        
        return output

# 建立關係網路
rn = RelationNetwork(
    object_dim=8,
    query_dim=4,
    g_hidden_dims=[32, 32, 32],
    f_hidden_dims=[64, 32],
    output_dim=10  # 例如 10 個答案類別
)

# 用範例物件測試
test_objects = [np.random.randn(8) for _ in range(5)]
test_query = np.random.randn(4)

output = rn.forward(test_objects, test_query)
print(f"\n關係網路輸出：{output[:5]}...")
print(f"輸出形狀：{output.shape}")

## Sort-of-CLEVR 資料集

使用彩色形狀的簡化視覺推理任務

In [ ]:
class SortOfCLEVR:
    """生成 Sort-of-CLEVR 資料集"""
    def __init__(self):
        self.colors = ['red', 'blue', 'green', 'orange', 'yellow', 'purple']
        self.shapes = ['circle', 'square', 'triangle']
        self.sizes = ['small', 'large']
    
    def generate_scene(self, n_objects=6):
        """
        生成包含物件的場景
        每個物件：(x, y, color_idx, shape_idx, size_idx)
        """
        objects = []
        used_colors = set()
        
        for i in range(n_objects):
            # 隨機位置
            x = np.random.uniform(0, 1)
            y = np.random.uniform(0, 1)
            
            # 唯一顏色
            available_colors = [c for c in range(len(self.colors)) if c not in used_colors]
            if not available_colors:
                break
            color_idx = np.random.choice(available_colors)
            used_colors.add(color_idx)
            
            # 隨機形狀和大小
            shape_idx = np.random.randint(len(self.shapes))
            size_idx = np.random.randint(len(self.sizes))
            
            objects.append({
                'x': x,
                'y': y,
                'color': color_idx,
                'shape': shape_idx,
                'size': size_idx
            })
        
        return objects
    
    def generate_question(self, scene, question_type='relational'):
        """
        生成問題：
        - 非關係型：「紅色物件的形狀是什麼？」
        - 關係型：「離紅色物件最近的物件的形狀是什麼？」
        """
        if question_type == 'relational':
            # 選擇一個參考物件
            ref_obj = np.random.choice(scene)
            
            # 找到最近的物件
            min_dist = float('inf')
            closest_obj = None
            for obj in scene:
                if obj is ref_obj:
                    continue
                dist = np.sqrt((obj['x'] - ref_obj['x'])**2 + (obj['y'] - ref_obj['y'])**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_obj = obj
            
            question = f"離 {self.colors[ref_obj['color']]} 最近的物件的形狀？"
            answer = closest_obj['shape']
            
        else:  # 非關係型
            # 選擇一個隨機物件
            obj = np.random.choice(scene)
            question = f"{self.colors[obj['color']]} 物件的形狀是什麼？"
            answer = obj['shape']
        
        return question, answer, question_type

# 生成範例場景
dataset = SortOfCLEVR()
scene = dataset.generate_scene(n_objects=6)

print("生成的場景：")
for i, obj in enumerate(scene):
    print(f"  物件 {i}：{dataset.colors[obj['color']]:8s} "
          f"{dataset.shapes[obj['shape']]:8s} {dataset.sizes[obj['size']]:6s} "
          f"在 ({obj['x']:.2f}, {obj['y']:.2f})")

# 生成問題
print("\n範例問題：")
for qtype in ['non-relational', 'relational', 'relational']:
    q, a, t = dataset.generate_question(scene, qtype)
    print(f"  [{t:15s}] {q}")
    print(f"  答案：{dataset.shapes[a]}")

## 視覺化場景

In [ ]:
def visualize_scene(scene, dataset):
    """視覺化 Sort-of-CLEVR 場景"""
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # 顏色映射
    color_map = {
        'red': 'red',
        'blue': 'blue',
        'green': 'green',
        'orange': 'orange',
        'yellow': 'yellow',
        'purple': 'purple'
    }
    
    for obj in scene:
        x, y = obj['x'], obj['y']
        color = color_map[dataset.colors[obj['color']]]
        shape = dataset.shapes[obj['shape']]
        size = 300 if obj['size'] == 1 else 150
        
        if shape == 'circle':
            ax.scatter([x], [y], s=size, c=color, marker='o', edgecolors='black', linewidths=2)
        elif shape == 'square':
            ax.scatter([x], [y], s=size, c=color, marker='s', edgecolors='black', linewidths=2)
        else:  # triangle
            ax.scatter([x], [y], s=size, c=color, marker='^', edgecolors='black', linewidths=2)
    
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.1, 1.1)
    ax.set_aspect('equal')
    ax.set_title('Sort-of-CLEVR 場景', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.show()

visualize_scene(scene, dataset)

## 物件表示編碼器

In [ ]:
def encode_object(obj, dataset):
    """
    將物件編碼為向量：
    [x, y, color_one_hot, shape_one_hot, size_one_hot]
    """
    # 位置
    pos = np.array([obj['x'], obj['y']])
    
    # One-hot 編碼
    color_oh = np.zeros(len(dataset.colors))
    color_oh[obj['color']] = 1
    
    shape_oh = np.zeros(len(dataset.shapes))
    shape_oh[obj['shape']] = 1
    
    size_oh = np.zeros(len(dataset.sizes))
    size_oh[obj['size']] = 1
    
    # 串接
    encoding = np.concatenate([pos, color_oh, shape_oh, size_oh])
    return encoding

def encode_question(question_text, ref_color, dataset):
    """
    將問題編碼為向量（簡化）
    實際上：使用 LSTM 或嵌入
    """
    # 參考顏色的 one-hot
    color_oh = np.zeros(len(dataset.colors))
    if ref_color is not None:
        color_oh[ref_color] = 1
    
    # 問題類型（簡化：關係型為 1，非關係型為 0）
    is_relational = 1.0 if 'closest' in question_text or '最近' in question_text else 0.0
    
    return np.concatenate([color_oh, [is_relational]])

# 測試編碼
obj_encoding = encode_object(scene[0], dataset)
print(f"物件編碼形狀：{obj_encoding.shape}")
print(f"物件編碼：{obj_encoding}")

q_encoding = encode_question("離 red 最近的物件的形狀？", 0, dataset)
print(f"\n問題編碼形狀：{q_encoding.shape}")

## 完整流程：場景 → 物件 → RN → 答案

In [ ]:
# 建立具有正確維度的關係網路
object_dim = 2 + len(dataset.colors) + len(dataset.shapes) + len(dataset.sizes)
query_dim = len(dataset.colors) + 1

rn_visual = RelationNetwork(
    object_dim=object_dim,
    query_dim=query_dim,
    g_hidden_dims=[64, 64, 32],
    f_hidden_dims=[64, 32],
    output_dim=len(dataset.shapes)  # 預測形狀
)

# 編碼場景
encoded_objects = [encode_object(obj, dataset) for obj in scene]

# 生成問題
question, answer, qtype = dataset.generate_question(scene, 'relational')

# 從問題中提取參考顏色（簡化）
ref_color = None
for i, color in enumerate(dataset.colors):
    if color in question.lower():
        ref_color = i
        break

encoded_question = encode_question(question, ref_color, dataset)

# 執行關係網路
prediction = rn_visual.forward(encoded_objects, encoded_question)
predicted_shape = np.argmax(prediction)

print(f"問題：{question}")
print(f"真實答案：{dataset.shapes[answer]}")
print(f"預測答案：{dataset.shapes[predicted_shape]}")
print(f"\n（模型未訓練，所以是隨機預測）")

## 視覺化物件之間的關係

In [ ]:
# 計算成對距離（關係的例子）
n_objects = len(scene)
distance_matrix = np.zeros((n_objects, n_objects))

for i in range(n_objects):
    for j in range(n_objects):
        dist = np.sqrt((scene[i]['x'] - scene[j]['x'])**2 + 
                      (scene[i]['y'] - scene[j]['y'])**2)
        distance_matrix[i, j] = dist

# 視覺化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 帶連接的場景
color_map = {'red': 'red', 'blue': 'blue', 'green': 'green', 
            'orange': 'orange', 'yellow': 'yellow', 'purple': 'purple'}

for i, obj_i in enumerate(scene):
    for j, obj_j in enumerate(scene):
        if i != j:
            # 繪製連接（越近越粗）
            dist = distance_matrix[i, j]
            alpha = np.exp(-dist * 2)  # 越近的物件 = 越高的 alpha
            ax1.plot([obj_i['x'], obj_j['x']], [obj_i['y'], obj_j['y']], 
                    'k-', alpha=alpha, linewidth=1)

for obj in scene:
    color = color_map[dataset.colors[obj['color']]]
    ax1.scatter([obj['x']], [obj['y']], s=300, c=color, 
               edgecolors='black', linewidths=3, zorder=5)
    ax1.text(obj['x'], obj['y']-0.08, dataset.colors[obj['color']], 
            ha='center', fontsize=9, fontweight='bold')

ax1.set_xlim(-0.1, 1.1)
ax1.set_ylim(-0.2, 1.1)
ax1.set_aspect('equal')
ax1.set_title('物件關係（空間）', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 距離矩陣
im = ax2.imshow(distance_matrix, cmap='viridis')
ax2.set_xlabel('物件', fontsize=12)
ax2.set_ylabel('物件', fontsize=12)
ax2.set_title('成對距離', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax2, label='距離')

plt.tight_layout()
plt.show()

print(f"\n關係網路考慮所有 {n_objects * (n_objects - 1)} 對！")

## 排列不變性測試

In [ ]:
# 測試 RN 對物件順序不變
test_objects = [np.random.randn(object_dim) for _ in range(4)]
test_query = np.random.randn(query_dim)

# 原始順序
output1 = rn_visual.forward(test_objects, test_query)

# 打亂順序
shuffled_objects = test_objects.copy()
np.random.shuffle(shuffled_objects)
output2 = rn_visual.forward(shuffled_objects, test_query)

# 檢查輸出是否相同
diff = np.linalg.norm(output1 - output2)

print("排列不變性測試：")
print(f"原始輸出：{output1[:4]}...")
print(f"打亂輸出：{output2[:4]}...")
print(f"差異：{diff:.10f}")
print(f"\n{'✓ 通過' if diff < 1e-10 else '✗ 失敗'}：RN 是排列不變的！")

## 與基線比較（無關係推理）

In [ ]:
class BaselineNetwork:
    """
    基線：只是串接所有物件 + 查詢，無明確關係
    """
    def __init__(self, object_dim, query_dim, max_objects, output_dim):
        # 串接所有物件 + 查詢
        input_dim = object_dim * max_objects + query_dim
        self.mlp = MLP(input_dim, [128, 64], output_dim)
        self.max_objects = max_objects
        self.object_dim = object_dim
    
    def forward(self, objects, query):
        # 填充或截斷到 max_objects
        padded = []
        for i in range(self.max_objects):
            if i < len(objects):
                padded.append(objects[i])
            else:
                padded.append(np.zeros(self.object_dim))
        
        # 串接所有東西
        concat = np.concatenate(padded + [query])
        return self.mlp.forward(concat)

# 建立基線
baseline = BaselineNetwork(object_dim, query_dim, max_objects=10, output_dim=len(dataset.shapes))

# 測試
baseline_output = baseline.forward(encoded_objects, encoded_question)

print("基線網路（無明確關係）：")
print(f"輸出：{baseline_output}")
print(f"\n基線沒有明確推理成對關係！")

## 關鍵要點

### 關係網路（RN）公式：

$$
\text{RN}(O) = f_\phi \left( \sum_{i,j} g_\theta(o_i, o_j, q) \right)
$$

其中：
- $O = \{o_1, o_2, ..., o_n\}$：物件集合
- $g_\theta$：關係函數（MLP）- 推理成對
- $f_\phi$：聚合函數（MLP）- 組合關係
- $q$：查詢/上下文（例如問題）

### 關鍵特性：

1. **明確的成對關係**：
   - 考慮所有 $n^2$ 對（或 $\binom{n}{2}$ 個唯一對）
   - 每對由 $g_\theta$ 獨立處理

2. **排列不變性**：
   - 求和聚合 → 順序不重要
   - $\text{RN}(\{o_1, o_2\}) = \text{RN}(\{o_2, o_1\})$

3. **組合性**：
   - 可以插入任何架構
   - 物件來自 CNN、LSTM 等

### 架構細節：

**用於視覺問答**：
```
圖像 → CNN → 特徵圖 → 物件（空間位置）
問題 → LSTM → 查詢嵌入
物件 + 查詢 → RN → 答案
```

**用於文本**：
```
句子 → LSTM → 詞嵌入 → 物件
查詢 → 嵌入
物件 + 查詢 → RN → 答案
```

### 計算複雜度：

- **成對數**：$O(n^2)$，其中 $n$ = 物件數
- **$g_\theta$ 評估次數**：$n^2$ 次前向傳遞
- 對於大 $n$ 可能很昂貴
- 可以使用 $i \neq j$ 排除自對 → $n(n-1)$ 對

### 結果：

**Sort-of-CLEVR**：
- 關係問題：96%（RN）vs 63%（CNN 基線）
- 非關係問題：98%（RN）vs 98%（CNN）

**CLEVR**（完整資料集）：
- 95.5% 準確率（超人類表現！）
- 之前最佳：68.5%

**bAbI**：
- 單一模型完成 18/20 個任務
- 在關係推理任務上表現強勁

### 為什麼有效：

1. **歸納偏置**：明確建模關係
2. **資料效率**：結構化計算 → 需要更少資料
3. **可解釋性**：可以視覺化 $g_\theta$ 輸出
4. **泛化**：學習關係模式

### 與其他方法的比較：

| 方法 | 成對關係 | 排列不變 | 複雜度 |
|------|---------|---------|--------|
| CNN | 隱式 | ✗ | $O(n)$ |
| RNN/LSTM | 序列 | ✗ | $O(n)$ |
| 注意力 | 加權對 | ✓ | $O(n^2)$ |
| **RN** | **明確** | **✓** | **$O(n^2)$** |
| 圖神經網路 | 明確（邊） | ✓ | $O(|E|)$ |

### 擴展：

- **自注意力**：具有可學習聚合的 RN 特例
- **Transformers**：注意力 = 關係推理！
- **圖神經網路**：圖結構上的 RN
- **關係 LSTM**：RN + 遞迴

### 限制：

- $O(n^2)$ 複雜度（對大 $n$ 昂貴）
- 求和聚合可能丟失資訊
- 需要物件提取（對圖像不簡單）

### 應用：

- 視覺問答
- 物理預測
- 多智能體系統
- 圖推理
- 關係資料庫
- 任何具有結構化物件的任務！